In [1]:
import pandas as pd
from pandas.stats.api import ols
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import math
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.cross_validation import train_test_split
import math

from sklearn.cross_validation import cross_val_score, cross_val_predict
from sklearn import metrics

filepath = '/Users/eshwarchandrasekharan/Desktop/repo/predict-pop/models-links/'
train_df = pd.read_csv(filepath + 'radshift-links-previous-to-next-page-info.csv')

one_hour_features = [
       'consumptions_by_type__link_clicks',
       'consumptions_by_type__other_clicks',
       'reactions_like_total',
       'stories_by_action_type__comment',
       'stories_by_action_type__like',
       'stories_by_action_type__share', 
        'engaged_fan',
       'fan_reach', 
    'impressions',
       'impressions_fan',
                ]

two_day_features = [
       'twoday_consumptions_by_type__link_clicks',
       'twoday_stories_by_action_type__share',
]

all_pages = train_df['page'].unique()
cross_promotion_features = all_pages

# train_features = one_hour_features
# print("ONLY 1 HOUR PERFORMANCE IN PREVIOUS PAGE")
# print('SHARES:')
# res = ols(y = train_df['next_shares'], x = train_df[train_features])
# print(res)
# print('CLICKS:')
# res = ols(y = train_df['next_clicks'], x = train_df[train_features])
# print(res)
# train_features = one_hour_features + page_infos# + cross_promotion_features
# for feat in cross_promotion_features:
#     train_features.append(feat)
# train_features

/Users/eshwarchandrasekharan/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# from sklearn import linear_model
# clf = linear_model.LinearRegression()
# print("LINEAR REGRESSION!")

from sklearn.tree import DecisionTreeRegressor
clf = DecisionTreeRegressor(max_depth = 10)
print("TREE REGRESSION!")

# train_df = pd.read_csv('radshift-links-previous-to-next-page-info.csv')
# all_pages = list(train_df.page.unique())

print("Performance stats with page names, and also cross-promotion order included!")
# print("WITH cross-promotion features")

page_infos = ['page', 'next_page']

# train_features = one_hour_features
# train_features = one_hour_features + page_infos + cross_promotion_features
train_features = one_hour_features + page_infos
# train_features = cross_promotion_features + one_hour_features

# log_scale = 1
log_scale = 0

if log_scale == 1:
    for feat in one_hour_features:
        train_df[feat] = np.log(train_df[feat] + 1)

# cross_promote = 0
cross_promote = 1

if cross_promote == 1:
    for feat in cross_promotion_features:
        train_features.append(feat)
        
# train_features = two_day_features + one_hour_features + cross_promotion_features

# for feats in (one_hour_features):
#     train_df[feats] = np.log(train_df[feats] + 1)

# lm = linear_model.LinearRegression(fit_intercept=True, normalize=True)
cv = 10

train_df['clicks_bucket'] = np.log(train_df['next_clicks'] + 1)
train_df['share_bucket'] = np.log(train_df['next_shares']+1)
# train_df['clicks_bucket'] = np.log(train_df['next_clicks'] + 1).astype(int)
# train_df['share_bucket'] = np.log(train_df['next_shares']+1).astype(int)

print("No. of data-points = ", len(train_df))

if log_scale == 0:
    print("Raw values!")
    y_clicks = train_df['next_clicks']
    y_share = train_df['next_shares']
else:
    print("Log scaled!")
    y_clicks = train_df['clicks_bucket']
    y_share = train_df['share_bucket']

###generate DFs for analysis - X and Y
X = train_df[train_features]
####convert page_infos into dummies
X = pd.get_dummies(data = X, columns=['page', 'next_page'])

from sklearn.model_selection import KFold

fold = 0
cv_folds = 10
kf = KFold(n_splits = cv_folds, shuffle = True)

clicks_accuracy = []
clicks_error = []
shares_accuracy = []
shares_error = []

share_cv_y_test = []
share_cv_y_pred = []
clicks_cv_y_test = []
clicks_cv_y_pred = []

for train_index, test_index in kf.split(X):
#     print("Fold = ", fold)
#     print("Shares: Fold = ", fold)
    y = y_share
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf.fit(X, y)
    y_pred = clf.predict(X_test)
    
    ####
    for pt in y_test:
        share_cv_y_test.append(pt)
    for pt in y_pred:
        share_cv_y_pred.append(pt)
        
    accuracy = metrics.r2_score(y_test, y_pred)
#     print("Cross-Predicted Accuracy (R2):", accuracy)
    shares_accuracy.append(accuracy)
    from sklearn.metrics import mean_absolute_error
    # print("Mean Absolute Error: ", mean_absolute_error(y, predictions))
    error_percent = mean_absolute_error(y_test, y_pred)/y.mean()
#     print("Mean values (share): ", y_test.mean(), " | percent error: ",  error_percent)
    shares_error.append(error_percent)
    
#     print("Clicks: Fold = ", fold)
    y = y_clicks
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf.fit(X, y)
    y_pred = clf.predict(X_test)
    
    ####
    for pt in y_test:
        clicks_cv_y_test.append(pt)
    for pt in y_pred:
        clicks_cv_y_pred.append(pt)

    accuracy = metrics.r2_score(y_test, y_pred)
#     print("Cross-Predicted Accuracy (R2):", accuracy)
    clicks_accuracy.append(accuracy)
    from sklearn.metrics import mean_absolute_error
    # print("Mean Absolute Error: ", mean_absolute_error(y, predictions))
    error_percent = mean_absolute_error(y_test, y_pred)/y.mean()
#     print("Mean values (share): ", y_test.mean(), " | percent error: ",  error_percent)
    clicks_error.append(error_percent)
    fold += 1
print("Shares performance: Accuracy = ", np.mean(shares_accuracy), " ; Error (/100) = ", np.mean(shares_error))
print("Clicks performance: Accuracy = ", np.mean(clicks_accuracy), " ; Error (/100) = ", np.mean(clicks_error))

if log_scale == 1:
    ##share
    print("Converting log-scale values back to exponents!")
    print('Shares!')
    accuracy = metrics.r2_score(np.exp(share_cv_y_test)-1, np.exp(share_cv_y_pred)-1)
    print("R2: ", accuracy)
    error_percent = mean_absolute_error(np.exp(share_cv_y_test)-1, np.exp(share_cv_y_pred)-1)/np.mean(np.exp(share_cv_y_test)-1)
    print("Error (\\100): ", error_percent)
    print("Mean: ", np.mean(np.exp(share_cv_y_test)-1))
#     plt.scatter(share_cv_y_test, share_cv_y_pred)

    print('Clicks!')
    accuracy = metrics.r2_score(np.exp(clicks_cv_y_test)-1, np.exp(clicks_cv_y_pred)-1)
    print("R2: ", accuracy)
    error_percent = mean_absolute_error(np.exp(clicks_cv_y_test)-1, np.exp(clicks_cv_y_pred)-1)/np.mean(np.exp(clicks_cv_y_test)-1)
    print("Error (\\100): ", error_percent)
    print("Mean: ", np.mean(np.exp(clicks_cv_y_test)-1))
#     plt.scatter(clicks_cv_y_test, clicks_cv_y_pred)

TREE REGRESSION!
Performance stats with page names, and also cross-promotion order included!
No. of data-points =  38769
Raw values!
Shares performance: Accuracy =  0.795540873589  ; Error (/100) =  0.564984989545
Clicks performance: Accuracy =  0.802176445372  ; Error (/100) =  0.42019896698


In [3]:
# Task 3: Routing!
# 1) 
# TREE REGRESSION!
# Performance stats with page names, and also cross-promotion order included!
# No. of data-points =  38769
# Raw values!
# Shares performance: Accuracy =  0.793776564986  ; Error (/100) =  0.564984135071
# Clicks performance: Accuracy =  0.803486044908  ; Error (/100) =  0.420198927612
    
# ----
# 2)
# TREE REGRESSION!
# Performance stats with page names, and also cross-promotion order included!
# No. of data-points =  38769
# Log scaled!
# Shares performance: Accuracy =  0.6575787588  ; Error (/100) =  0.212635134964
# Clicks performance: Accuracy =  0.721635998508  ; Error (/100) =  0.0910451690249
    
# 2.1) Convert log-scale predictions/test to exponents and compute R2/errors

# Shares!
# R2:  0.779841664501
# Error (\100):  0.516329728148
# Mean:  118.348990173
# Clicks!
# R2:  0.779658862269
# Error (\100):  0.398451498565
# Mean:  10139.2730016
# ----

In [3]:
###Get all links where "next_page" == test_page, and then sort links based on predicted shares/clicks. 
###Verify how many of the sorted links are in the right place, wrt to actual values of shares/clicks!
test_page = 'Tasty'
test_df = train_df[train_df.next_page == test_page]

In [4]:
one_hour_features = [
       'consumptions_by_type__link_clicks',
       'consumptions_by_type__other_clicks',
       'reactions_like_total',
       'stories_by_action_type__comment',
       'stories_by_action_type__like',
       'stories_by_action_type__share', 
        'engaged_fan',
       'fan_reach', 
    'impressions',
       'impressions_fan',
                ]

two_day_features = [
       'twoday_consumptions_by_type__link_clicks',
       'twoday_stories_by_action_type__share',
]

all_pages = train_df['page'].unique()
cross_promotion_features = all_pages

page_infos = ['page', 'next_page']

# train_features = one_hour_features
# train_features = one_hour_features + page_infos + cross_promotion_features
test_features = one_hour_features + page_infos
# train_features = cross_promotion_features + one_hour_features

# cross_promote = 0
cross_promote = 1

if cross_promote == 1:
    for feat in cross_promotion_features:
        test_features.append(feat)

# log_scale = 1
log_scale = 0

if log_scale == 1:
    for feat in one_hour_features:
        test_df[feat] = np.log(test_df[feat] + 1)

In [5]:
train_df['next_page'].unique().shape

(75,)

In [6]:
#####train on all, test per page! 

pages = []
accuracy = []
topK_hit_rate = []
num_links = []

print("Shares!")
# test_page = 'Tasty'

test_pages = [
'BuzzFeed Food',
'BuzzFeed',
'BuzzFeed Video',
'Buy Me That',
'BuzzFeed Parents',
'BuzzFeed Australia',
'BuzzFeed News',
'BuzzFeed Entertainment',
'BuzzFeed UK News',
'BuzzFeed Celeb',
'Tasty',
'BuzzFeed Geeky',
'BuzzFeed Health',
'BuzzFeed DIY',
'SOML',
'BuzzFeed Books',
'BuzzFeed Style',
'BuzzFeed UK',
'Obsessed by BuzzFeed',
'BuzzFeed Animals',
'Cocoa Butter',
'BuzzFeed Unsolved',
'BuzzFeed Science',
'Cheeky',
'BuzzFeed Quiz',
'BuzzFeed Politics',
'BuzzFeed LGBT',
'BuzzFeed Weddings',
'Bien Tasty',
'Top Knot',
'BuzzFeed India',
'Reasons to Smile',
'BuzzFeed World',
'LOLA',
'Proper Tasty',
'Nifty',
'BuzzFeed Tech',
'Tasty One-Pot',
'BuzzFeed Sweaty',
]

for test_page in test_pages:
# for test_page in train_df['next_page'].unique():
# for test_page in ['Tasty', 'SOML', 'BuzzFeed News', 'Nifty', 'BuzzFeed Animals']:
#     print("Test page: ", test_page)
    X = train_df
    # X = train_df[~(train_df.next_page == test_page)]
    y_train = X['next_shares']
    X = X[test_features]
    X['test_split'] = 0

    test_df = train_df[train_df.next_page == test_page]
    if len(test_df) < 5:
        print("Skipping... Cos < 5 links on ", test_page)
        continue
    num_links.append(len(test_df))
    y_test = test_df[['next_shares', 'external_id']]
    test_df = test_df[test_features]
    test_df['test_split'] = 1

    X = pd.concat([X,test_df])

    ####convert page_infos into dummies
    X = pd.get_dummies(data = X, columns=['page', 'next_page'])

    X_train = X[X.test_split == 0]
    X_test = X[X.test_split == 1]

    from sklearn.tree import DecisionTreeRegressor
    clf = DecisionTreeRegressor(max_depth = 10)
#     print("TREE REGRESSION!")

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

#     accuracy = metrics.r2_score(y_test['next_shares'], y_pred)
#     print("Cross-Predicted Accuracy (R2):", accuracy)
#     from sklearn.metrics import mean_absolute_error
#     print("Mean Absolute Error: ", mean_absolute_error(y_test['next_shares'], y_pred))
#     error_percent = mean_absolute_error(y_test['next_shares'], y_pred)/y_test['next_shares'].mean()
#     print("Mean values (share): ", y_test['next_shares'].mean(), " | percent error: ",  error_percent)

    ###sort and see how many pair-wise predictions are correct!?
    order = pd.DataFrame()
    order = y_test
    # order['actual_shares'] = y_test
    order['pred_shares'] = y_pred

    order['actual_rank'] = order['next_shares'].rank(ascending=0)
    order['predicted_rank'] = order['pred_shares'].rank(ascending=0)

    ###randomly pick 2 external_ids and make predictions on which will do better than the other:
    limit = 10000
    hits = 0
    miss = 0 
    total = 0
    
    for i in range(1,limit):
        total += 1
        trial = order.sample(n=2)
    #     print(trial)
        act1 = trial[:1].next_shares.values[0]
        act2 = trial[1:2].next_shares.values[0]
        pred1 = trial[:1].pred_shares.values[0]
        pred2 = trial[1:2].pred_shares.values[0]

        if act1 >= act2:
            if pred1 >= pred2:
                hits += 1
            else:
                miss += 1
        else:
            if pred1 < pred2:
                hits += 1
            else:
                miss += 1
    # test = test_df.sample(n = 2)
    print("Test page:", test_page, "; Accuracy: ", hits/total, "; Total: ", total, "; Hits: ", hits, "; Miss: ", miss)
#     print("Test page: ", test_page, "; Accuracy: ", hits/total)
    pages.append(test_page)
    accuracy.append(hits/total)
    ###top K predictions
    top_k = 20
    temp = order.sort_values('actual_rank', ascending = True)[:top_k]
    hits = len(temp[temp['predicted_rank'] <= top_k])
    print('Total links: ', len(test_df), '; Top K: ', top_k, '; Hits: ', hits, "; hit-rate: ", hits/top_k)
    topK_hit_rate.append(hits/top_k)

simulations = pd.DataFrame()
simulations['page'] = pages
simulations['accuracy'] = accuracy
simulations['topK_hit_rate'] = topK_hit_rate
simulations['total_links'] = num_links

print("DONE!")

Shares!


/Users/eshwarchandrasekharan/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Test page: BuzzFeed Food ; Accuracy:  0.7542754275427542 ; Total:  9999 ; Hits:  7542 ; Miss:  2457
Total links:  922 ; Top K:  20 ; Hits:  11 ; hit-rate:  0.55
Test page: BuzzFeed ; Accuracy:  0.7726772677267727 ; Total:  9999 ; Hits:  7726 ; Miss:  2273
Total links:  7547 ; Top K:  20 ; Hits:  16 ; hit-rate:  0.8
Test page: BuzzFeed Video ; Accuracy:  0.8030803080308031 ; Total:  9999 ; Hits:  8030 ; Miss:  1969
Total links:  2160 ; Top K:  20 ; Hits:  13 ; hit-rate:  0.65
Test page: Buy Me That ; Accuracy:  0.6707670767076708 ; Total:  9999 ; Hits:  6707 ; Miss:  3292
Total links:  595 ; Top K:  20 ; Hits:  4 ; hit-rate:  0.2
Test page: BuzzFeed Parents ; Accuracy:  0.7075707570757076 ; Total:  9999 ; Hits:  7075 ; Miss:  2924
Total links:  459 ; Top K:  20 ; Hits:  5 ; hit-rate:  0.25
Test page: BuzzFeed Australia ; Accuracy:  0.6788678867886788 ; Total:  9999 ; Hits:  6788 ; Miss:  3211
Total links:  3141 ; Top K:  20 ; Hits:  2 ; hit-rate:  0.1
Test page: BuzzFeed News ; Accuracy

In [20]:
simulations.to_csv('task3-simulations.csv', index = False)
simulations

,page,accuracy,topK_hit_rate,total_links
0,BuzzFeed Food,0.762676,0.55,922
1,BuzzFeed,0.770477,0.80,7547
2,BuzzFeed Video,0.804280,0.65,2160
3,Buy Me That,0.670367,0.20,595
4,BuzzFeed Parents,0.704870,0.25,459
5,BuzzFeed Australia,0.687269,0.10,3141
6,BuzzFeed News,0.689869,0.30,2234
7,BuzzFeed Entertainment,0.717572,0.35,1505
8,BuzzFeed UK News,0.657966,0.30,320
9,BuzzFeed Celeb,0.708071,0.35,1096


In [307]:
###randomly pick 2 external_ids and make predictions on which will do better than the other:

limit =10000
hits = 0
miss = 0 
total = 0

for i in range(1,limit):
    total += 1
    trial = order[:100].sample(n=2)
#     print(trial)
    act1 = trial[:1].next_shares.values[0]
    act2 = trial[1:2].next_shares.values[0]
    pred1 = trial[:1].pred_shares.values[0]
    pred2 = trial[1:2].pred_shares.values[0]
    
    if act1 >= act2:
        if pred1 >= pred2:
            hits += 1
        else:
            miss += 1
    else:
        if pred1 < pred2:
            hits += 1
        else:
            miss += 1
# test = test_df.sample(n = 2)
print("Total: ", total, "; Hits: ", hits, "; Miss: ", miss)
print("Accuracy: ", hits/total)

Total:  9999 ; Hits:  8330 ; Miss:  1669
Accuracy:  0.8330833083308331


In [242]:
# Randomly pick 2 external_ids and make predictions on which will do better than the other:
# Total:  9999 ; Hits:  8462 ; Miss:  1537
# Accuracy: 0.8462846284628462

In [243]:
###randomly pick 2 external_ids from TOP 100 and make predictions on which will do better than the other:

limit = 100
hits = 0
miss = 0 
total = 0

for i in range(1,limit):
    total += 1
    trial = order[:100].sample(n=2)
#     print(trial)
    act1 = trial[:1].next_shares.values[0]
    act2 = trial[1:2].next_shares.values[0]
    pred1 = trial[:1].pred_shares.values[0]
    pred2 = trial[1:2].pred_shares.values[0]
    
    if act1 >= act2:
        if pred1 >= pred2:
            hits += 1
        else:
            miss += 1
    else:
        if pred1 < pred2:
            hits += 1
        else:
            miss += 1
# test = test_df.sample(n = 2)
print("Total: ", total, "; Hits: ", hits, "; Miss: ", miss)
print("Accuracy: ", hits/total)

Total:  99 ; Hits:  86 ; Miss:  13
Accuracy:  0.8686868686868687


In [244]:
###CLICKS

In [299]:
#####train on all, test per page! 

print("CLICKS!")
X = train_df[test_features]
y_train = train_df['next_clicks']

X['test_split'] = 0

test_page = 'SOML'
# test_page = 'Tasty'
test_df = train_df[train_df.next_page == test_page]
y_test = test_df[['next_clicks', 'external_id']]
test_df = test_df[test_features]
test_df['test_split'] = 1

X = pd.concat([X,test_df])

####convert page_infos into dummies
X = pd.get_dummies(data = X, columns=['page', 'next_page'])

X_train = X[X.test_split == 0]
X_test = X[X.test_split == 1]

from sklearn.tree import DecisionTreeRegressor
clf = DecisionTreeRegressor(max_depth = 10)
print("TREE REGRESSION!")

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

accuracy = metrics.r2_score(y_test['next_clicks'], y_pred)
print("Cross-Predicted Accuracy (R2):", accuracy)
from sklearn.metrics import mean_absolute_error
print("Mean Absolute Error: ", mean_absolute_error(y_test['next_clicks'], y_pred))
error_percent = mean_absolute_error(y_test['next_clicks'], y_pred)/y_test['next_clicks'].mean()
print("Mean values (share): ", y_test['next_clicks'].mean(), " | percent error: ",  error_percent)

###sort and see how many pair-wise predictions are correct!?
order = pd.DataFrame()
order = y_test
# order['actual_shares'] = y_test
order['pred_clicks'] = y_pred

order['actual_rank'] = order['next_clicks'].rank(ascending=0)
order['predicted_rank'] = order['pred_clicks'].rank(ascending=0)

CLICKS!


/Users/eshwarchandrasekharan/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


TREE REGRESSION!
Cross-Predicted Accuracy (R2): 0.0451870042936
Mean Absolute Error:  3746.0814707
Mean values (share):  4882.68537414966  | percent error:  0.767217459994


In [248]:
###randomly pick 2 external_ids and make predictions on which will do better than the other: wrt CLICKS

limit =100000
hits = 0
miss = 0 
total = 0

for i in range(1,limit):
    total += 1
    trial = order[:100].sample(n=2)
#     print(trial)
    act1 = trial[:1].next_clicks.values[0]
    act2 = trial[1:2].next_clicks.values[0]
    pred1 = trial[:1].pred_clicks.values[0]
    pred2 = trial[1:2].pred_clicks.values[0]
    
    if act1 >= act2:
        if pred1 >= pred2:
            hits += 1
        else:
            miss += 1
    else:
        if pred1 < pred2:
            hits += 1
        else:
            miss += 1
# test = test_df.sample(n = 2)
print("Total: ", total, "; Hits: ", hits, "; Miss: ", miss)
print("Accuracy: ", hits/total)

Total:  99999 ; Hits:  79901 ; Miss:  20098
Accuracy:  0.7990179901799018


In [247]:
###randomly pick 2 external_ids from TOP 100 and make predictions on which will do better than the other:

limit = 100
hits = 0
miss = 0 
total = 0

for i in range(1,limit):
    total += 1
    trial = order[:100].sample(n=2)
#     print(trial)
    act1 = trial[:1].next_clicks.values[0]
    act2 = trial[1:2].next_clicks.values[0]
    pred1 = trial[:1].pred_clicks.values[0]
    pred2 = trial[1:2].pred_clicks.values[0]
    
    if act1 >= act2:
        if pred1 >= pred2:
            hits += 1
        else:
            miss += 1
    else:
        if pred1 < pred2:
            hits += 1
        else:
            miss += 1
# test = test_df.sample(n = 2)
print("Total: ", total, "; Hits: ", hits, "; Miss: ", miss)
print("Accuracy: ", hits/total)

Total:  99 ; Hits:  72 ; Miss:  27
Accuracy:  0.7272727272727273


In [294]:
######### INCLUDE TITLE FEATURES ALSO!!!
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import scipy.sparse

# tfidf = Pipeline([
#     ('vectorizer', HashingVectorizer(ngram_range=(1,4), non_negative=True)),
#     ('tfidf', TfidfTransformer()),
# ])

title_vect = Pipeline([
                     ('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ])

X = train_df
y_train = train_df['next_shares']
X['test_split'] = 0

test_page = 'SOML'
# test_page = 'Nifty'
# test_page = 'Tasty'
test_df = train_df[train_df.next_page == test_page]
y_test = test_df[['next_shares', 'external_id']]
test_df['test_split'] = 1

X = pd.concat([X,test_df])
X = X[test_features + ['title', 'test_split']]
####convert page_infos into dummies
X = pd.get_dummies(data = X, columns=['page', 'next_page'])

X_train = X[X.test_split == 0]
X_test = X[X.test_split == 1]

###train
X_tfidf = title_vect.fit_transform(X_train['title'])
X_other = X_train.drop(['title', 'test_split'], axis = 1)
X_train = scipy.sparse.hstack([X_tfidf, X_other])

###test
X_tfidf = title_vect.transform(X_test['title'])
X_other = X_test.drop(['title', 'test_split'], axis = 1)
X_test = scipy.sparse.hstack([X_tfidf, X_other])

from sklearn.tree import DecisionTreeRegressor
clf = DecisionTreeRegressor(max_depth = 10)
print("TREE REGRESSION!")

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy = metrics.r2_score(y_test['next_shares'], y_pred)
print("Cross-Predicted Accuracy (R2):", accuracy)
from sklearn.metrics import mean_absolute_error
print("Mean Absolute Error: ", mean_absolute_error(y_test['next_shares'], y_pred))
error_percent = mean_absolute_error(y_test['next_shares'], y_pred)/y_test['next_shares'].mean()
print("Mean values (share): ", y_test['next_shares'].mean(), " | percent error: ",  error_percent)

###sort and see how many pair-wise predictions are correct!?
order = pd.DataFrame()
order = y_test
# order['actual_shares'] = y_test
order['pred_shares'] = y_pred

order['actual_rank'] = order['next_shares'].rank(ascending=0)
order['predicted_rank'] = order['pred_shares'].rank(ascending=0)

/Users/eshwarchandrasekharan/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


TREE REGRESSION!
Cross-Predicted Accuracy (R2): 0.0800118692307
Mean Absolute Error:  65.2914646141
Mean values (share):  81.15646258503402  | percent error:  0.804513436569


/Users/eshwarchandrasekharan/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/eshwarchandrasekharan/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/eshwarchandrasekharan/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [296]:
###randomly pick 2 external_ids and make predictions on which will do better than the other:
limit =10000
hits = 0
miss = 0 
total = 0

for i in range(1,limit):
    total += 1
    trial = order[:100].sample(n=2)
#     print(trial)
    act1 = trial[:1].next_shares.values[0]
    act2 = trial[1:2].next_shares.values[0]
    pred1 = trial[:1].pred_shares.values[0]
    pred2 = trial[1:2].pred_shares.values[0]
    
    if act1 >= act2:
        if pred1 >= pred2:
            hits += 1
        else:
            miss += 1
    else:
        if pred1 < pred2:
            hits += 1
        else:
            miss += 1
# test = test_df.sample(n = 2)
print("Total: ", total, "; Hits: ", hits, "; Miss: ", miss)
print("Accuracy: ", hits/total)

Total:  9999 ; Hits:  7605 ; Miss:  2394
Accuracy:  0.7605760576057605


In [298]:
test_page = 'SOML'
# test_page = 'Nifty'
# test_page = 'Tasty'
train_df[train_df.next_page == test_page].shape

(588, 111)

In [300]:
train_df.next_page.unique()

array(['BuzzFeed Food', 'BuzzFeed', 'BuzzFeed Video', 'Buy Me That',
       'BuzzFeed Philippines', 'BuzzFeed Parents', 'BuzzFeed Australia',
       'BuzzFeed News', 'BuzzFeed Scotland', 'BuzzFeed Entertainment',
       'BuzzFeed UK News', 'Quizzes En Español', 'BuzzFeed Celeb', 'Tasty',
       'BuzzFeed Geeky', 'BuzzFeed Health', 'BuzzFeed DIY', 'SOML',
       'BuzzFeed Rewind', 'BuzzFeed Books', 'BuzzFeed Style',
       'BuzzFeed UK', 'Obsessed by BuzzFeed', 'BuzzFeed Animals',
       'Cocoa Butter', 'BuzzFeed Unsolved', 'BuzzFeed Science', 'Cheeky',
       'BuzzFeed México', 'BuzzFeed Ladylike', 'BuzzFeed España',
       'BuzzFeed Quiz', 'BuzzFeed Brasil', 'BuzzFeed Español',
       'BuzzFeed Politics', 'BuzzFeed Oz Politics', 'BuzzFeed Japan',
       'Another Round', 'Pero Like', 'BuzzFeed Community',
       'BuzzFeed Canada', 'BuzzFeed Oz News', 'BuzzFeed LGBT',
       'BuzzFeed Weddings', 'BuzzFeed France', 'Bien Tasty',
       'BuzzFeed IRL', 'BuzzFeed UK Politics', 'Tasty Demai

In [ ]:
#####mode-report testing

In [24]:
#######FIT THE MODEL ON HISTORICAL DATA!!!

import pandas as pd
from pandas.stats.api import ols
from sklearn import datasets, linear_model
from matplotlib import pyplot as plt
import math
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
import math
from sklearn import metrics

filepath = '/Users/eshwarchandrasekharan/Desktop/repo/predict-pop/models-links/'
train_df = pd.read_csv(filepath + 'radshift-links-previous-to-next-page-info.csv')
# train_df = build_df
train_df = train_df.fillna(0)

one_hour_features = [
#        'consumptions_by_type__link_clicks',
#       'consumptions_by_type__other_clicks',
#       'reactions_like_total',
#       'stories_by_action_type__comment',
#       'stories_by_action_type__like',
       'stories_by_action_type__share', 
#         'engaged_fan',
#       'fan_reach', 
#     'impressions',
#       'impressions_fan',
                ]

cross_promotion_features = all_pages

from sklearn import linear_model
clf = linear_model.LinearRegression()
print("LINEAR REGRESSION!")

# from sklearn.tree import DecisionTreeRegressor
# clf = DecisionTreeRegressor(max_depth = 5)
# print("TREE REGRESSION!")

print("Performance stats with page names, and also cross-promotion order included!")
# print("WITH cross-promotion features")

page_infos = ['page', 'next_page']
# page_infos = ['display_name', 'next_page']   ###mode report variables

train_features = one_hour_features

# include_page_info = 0
include_page_info = 1
if include_page_info == 1:
  train_features = one_hour_features + page_infos
# train_features = cross_promotion_features + one_hour_features

log_scale = 1
# log_scale = 0

if log_scale == 1:
    for feat in one_hour_features:
        train_df[feat] = np.log(train_df[feat] + 1)

cross_promote = 0
# cross_promote = 1

if cross_promote == 1:
    for feat in cross_promotion_features:
        train_features.append(feat)

train_df['clicks_bucket'] = np.log(train_df['next_clicks'] + 1)
train_df['share_bucket'] = np.log(train_df['next_shares']+1)

print("No. of data-points = ", len(train_df))

###generate DFs for analysis - X and Y
X = train_df[train_features]
y = train_df['share_bucket']
# y = train_df['next_shares']
####convert page_infos into dummies
if include_page_info == 1:
#   X = pd.get_dummies(data = X, columns=['display_name', 'next_page']) ###mode report variables
  X = pd.get_dummies(data = X, columns=['page', 'next_page']) 

###test vector to be in this dimension space too!
test_features = X.columns

clf = clf.fit(X, y)
clf

LINEAR REGRESSION!
Performance stats with page names, and also cross-promotion order included!
No. of data-points =  38769


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [27]:
####make predictions on performance if posted on the target page, given the perfoprmance in the previous page!
# test_page = 'BuzzFeed Video'
test_page = 'BuzzFeed'
max_suggestions = 10
test_df = train_df
X_test = test_df[train_features]

###save for later!
# curr_page = X_test['page']

if include_page_info == 1:
#   X_test = pd.get_dummies(data = X_test, columns=['display_name', 'next_page'])
  X_test = pd.get_dummies(data = X_test, columns=['page', 'next_page'])

for page in test_df['next_page'].unique():
  X_test['next_page_' + page] = 0

X_test['next_page_' + test_page] = 1

test_df['predicted_shares'] = clf.predict(X_test)
test_df['predicted_shares'] = (np.exp(test_df['predicted_shares'])-1).astype(int)

result_cols = ['page', 'title', 'buzz_id', 'predicted_shares']
# result_cols = ['display_name', 'title', 'buzz_id', 'predicted_shares']
test_df = test_df[~(test_df['page'] == test_page)]
# test_df = test_df[~(test_df['display_name'] == test_page)]
# ###only keep candidates that are from pages which have sourced test_page in the past!
candidate_pages = test_df[test_df.next_page == test_page]['page'].unique()
test_df = test_df[test_df['page'].isin(candidate_pages)]
###
test_df = test_df[result_cols].drop_duplicates(subset=['buzz_id'], keep = 'last')
test_df.sort_values('predicted_shares', ascending = False)[:max_suggestions]

,page,title,buzz_id,predicted_shares
29785,Cheeky,Thousands Of Americans Crashed Canada's Immigr...,4393851.0,4505
18311,BuzzFeed Canada,This Is Already The Most Iconic Photo Of The T...,4465035.0,3776
31004,BuzzFeed Philippines,Anthony Bourdain's Manila Episode Of &quot;Par...,4240567.0,3259
26826,BuzzFeed News,Dakota Access Pipeline Will Be Rerouted In A V...,4412034.0,3150
26479,BuzzFeed News,Ohio Passes Bill That Bans Abortion After A Mo...,4414644.0,2650
32705,BuzzFeed UK News,Angelina Jolie Has Filed For Divorce From Brad...,4359699.0,2463
21471,BuzzFeed Philippines,17 Questions Filipino Catholic Schools Need To...,3795841.0,2288
38682,Cheeky,People Are Loving This Woman's Mirror Finish C...,4244831.0,2193
38014,Cheeky,"The ""Cheerio Challenge"" Is Proof We Are Living...",4281568.0,2034
18073,BuzzFeed Philippines,15 Times Filipinas Refused To Take Shit From T...,4439404.0,2027


In [33]:
print("Using title features too!")

Using title features too!


In [42]:
#######FIT THE MODEL ON HISTORICAL DATA!!!

import pandas as pd
from pandas.stats.api import ols
from sklearn import datasets, linear_model
from matplotlib import pyplot as plt
import math
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
import math
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import scipy.sparse

filepath = '/Users/eshwarchandrasekharan/Desktop/repo/predict-pop/models-links/'
train_df = pd.read_csv(filepath + 'radshift-links-previous-to-next-page-info.csv')
# train_df = build_df
train_df = train_df.fillna(0)

one_hour_features = [
#        'consumptions_by_type__link_clicks',
#       'consumptions_by_type__other_clicks',
#       'reactions_like_total',
#       'stories_by_action_type__comment',
#       'stories_by_action_type__like',
       'stories_by_action_type__share', 
#         'engaged_fan',
#       'fan_reach', 
#     'impressions',
#       'impressions_fan',
                ]

cross_promotion_features = all_pages

from sklearn import linear_model
clf = linear_model.LinearRegression()
print("LINEAR REGRESSION!")

# from sklearn.tree import DecisionTreeRegressor
# clf = DecisionTreeRegressor(max_depth = 5)
# print("TREE REGRESSION!")

print("Performance stats with page names, and also cross-promotion order included!")
# print("WITH cross-promotion features")

page_infos = ['page', 'next_page']
# page_infos = ['display_name', 'next_page']   ###mode report variables

train_features = one_hour_features

# include_page_info = 0
include_page_info = 1
if include_page_info == 1:
  train_features = one_hour_features + page_infos
# train_features = cross_promotion_features + one_hour_features

log_scale = 1
# log_scale = 0

if log_scale == 1:
    for feat in one_hour_features:
        train_df[feat] = np.log(train_df[feat] + 1)

cross_promote = 0
# cross_promote = 1

if cross_promote == 1:
    for feat in cross_promotion_features:
        train_features.append(feat)

train_df['clicks_bucket'] = np.log(train_df['next_clicks'] + 1)
train_df['share_bucket'] = np.log(train_df['next_shares']+1)

print("No. of data-points = ", len(train_df))

###generate DFs for analysis - X and Y
X = train_df[train_features + ['title']]
y = train_df['share_bucket']
# y = train_df['next_shares']
####convert page_infos into dummies
if include_page_info == 1:
#   X = pd.get_dummies(data = X, columns=['display_name', 'next_page']) ###mode report variables
  X = pd.get_dummies(data = X, columns=['page', 'next_page']) 

####include title features!
title_vect = Pipeline([
                     ('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ])

##
X_tfidf = title_vect.fit_transform(X['title'])
X = X.drop(['title'], axis = 1)
X = scipy.sparse.hstack([X_tfidf, X])

clf = clf.fit(X, y)
clf

LINEAR REGRESSION!
Performance stats with page names, and also cross-promotion order included!
No. of data-points =  38769


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [43]:
####make predictions on performance if posted on the target page, given the perfoprmance in the previous page!
# test_page = 'BuzzFeed Video'
test_page = 'BuzzFeed'
max_suggestions = 10
test_df = train_df
X_test = test_df[train_features + ['title']]

###save for later!
# curr_page = X_test['page']

if include_page_info == 1:
#   X_test = pd.get_dummies(data = X_test, columns=['display_name', 'next_page'])
  X_test = pd.get_dummies(data = X_test, columns=['page', 'next_page'])
    
for page in test_df['next_page'].unique():
  X_test['next_page_' + page] = 0

X_test['next_page_' + test_page] = 1

##title features
X_tfidf = title_vect.transform(X_test['title'])
X_test = X_test.drop(['title'], axis = 1)
X_test = scipy.sparse.hstack([X_tfidf, X_test])
###

test_df['predicted_shares'] = clf.predict(X_test)
test_df['predicted_shares'] = (np.exp(test_df['predicted_shares'])-1).astype(int)

result_cols = ['page', 'title', 'buzz_id', 'predicted_shares']
# result_cols = ['display_name', 'title', 'buzz_id', 'predicted_shares']
test_df = test_df[~(test_df['page'] == test_page)]
# test_df = test_df[~(test_df['display_name'] == test_page)]
# ###only keep candidates that are from pages which have sourced test_page in the past!
candidate_pages = test_df[test_df.next_page == test_page]['page'].unique()
test_df = test_df[test_df['page'].isin(candidate_pages)]
###
test_df = test_df[result_cols].drop_duplicates(subset=['buzz_id'], keep = 'last')
test_df.sort_values('predicted_shares', ascending = False)[:max_suggestions]

,page,title,buzz_id,predicted_shares
29785,Cheeky,Thousands Of Americans Crashed Canada's Immigr...,4393851.0,10232
26826,BuzzFeed News,Dakota Access Pipeline Will Be Rerouted In A V...,4412034.0,7528
38733,BuzzFeed Health,24 Things Only People With Shit Eyesight Will ...,4239651.0,6594
23993,BuzzFeed Entertainment,Debbie Reynolds Dies One Day After Daughter Ca...,4430695.0,4699
29245,BuzzFeed World,Two People Dead After Magnitude-7.8 Earthquake...,4397061.0,4601
13921,BuzzFeed UK News,Sandi Toksvig And Noel Fielding Are The New Ho...,4489251.0,3992
38747,BuzzFeed Celeb,Kristen Bell And Mila Kunis Being Surprised By...,4240936.0,3826
24204,BuzzFeed News,&quot;Star Wars&quot; Icon Carrie Fisher Dies,4428606.0,3761
23006,BuzzFeed Philippines,20 Things You'll Only Understand If You're Sli...,4130135.0,3583
30008,BuzzFeed Oz Politics,The Senate Has Killed Off The Marriage Equalit...,4391870.0,3306


In [51]:
test_df.sort_values('predicted_shares', ascending = False)[:max_suggestions]

,page,title,buzz_id,predicted_shares
29785,Cheeky,Thousands Of Americans Crashed Canada's Immigr...,4393851.0,3105
26826,BuzzFeed News,Dakota Access Pipeline Will Be Rerouted In A V...,4412034.0,2284
38733,BuzzFeed Health,24 Things Only People With Shit Eyesight Will ...,4239651.0,2001
23993,BuzzFeed Entertainment,Debbie Reynolds Dies One Day After Daughter Ca...,4430695.0,1425
29245,BuzzFeed World,Two People Dead After Magnitude-7.8 Earthquake...,4397061.0,1396
13921,BuzzFeed UK News,Sandi Toksvig And Noel Fielding Are The New Ho...,4489251.0,1211
38747,BuzzFeed Celeb,Kristen Bell And Mila Kunis Being Surprised By...,4240936.0,1160
24204,BuzzFeed News,&quot;Star Wars&quot; Icon Carrie Fisher Dies,4428606.0,1141
23006,BuzzFeed Philippines,20 Things You'll Only Understand If You're Sli...,4130135.0,1087
20725,BuzzFeed News,A Nationwide Ban On Abortions After A Month An...,4448375.0,1068
